In [ ]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils
from IPython.display import display

# Project imports 
from data import mnist_m as mnistm
from data import mnist
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from data.tasks import load_task
from experiments.training import *
#from experiments.SL_bound import *
#from experiments.DA_bound import *
from bounds.bounds import *
from util.kl import *
from util.mmd import *
from util.misc import *
from results.plotting import *

# Hyper-parameters
task = 4
architecture="lenet"
binary=True
#project_folder="/cephyr/users/adambre/Alvis/mnist_transfer/"
project_folder="/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/"
if binary:
    results_folder = project_folder+"results/task"+str(task)+"/Binary/"+str(architecture)
else:
    results_folder = project_folder+"results/task"+str(task)+"/"+str(architecture)

In [ ]:
r_files = [f for f in os.listdir(results_folder) if ('results.pkl' in f and not 'prior_results.pkl' in f)]

D = pd.DataFrame({})

# print(len(r_files))
# print(r_files)
# sys.exit(-1)
for r in r_files: 
    Dr = pd.read_pickle(os.path.join(results_folder, r))
    D = pd.concat([D, Dr], axis=0,sort=False)
#D.head(20)    
D['germain_bound'] = np.nan
D['beta_bound'] = np.nan
D['mmd_bound'] = np.nan

D = D.sort_values(by=[ 'alpha','Weightupdates', 'sigma'])#

D.head(70)

In [ ]:
m_bound = D['m_bound'].values[0]
m_target = D['m_target'].values[0]
delta = D['delta'].values[0]
"""
###### average over all seeds
new_df=pd.DataFrame({})

for a in [0,0.3]:#np.sort(D['alpha'].unique()):
    for s in np.sort(D['sigma'].unique()):
        for e in np.sort(D['epsilon'].unique()): 
            for w in np.sort(D['Weightupdates'].unique()):
                stratum = (D['sigma']==s)&(D['alpha']==a)&(D['epsilon']==e)&(D['Weightupdates']==w)
                if(len(D[stratum])>=2):
                    print(D[stratum])
sys.exit(-1)


### take 
"""
### have the value of beta_infinity ready to use for the final computation of the bounds
if task==2:
    beta_inf=10.986111
    MMD=1.41
elif task==6:
    beta_inf=4
    MMD=1.41 ##? 
else:
    beta_inf=-1
    
    ###calculate mmd outside of loop
#MMD=calculate_mmd(4)
#print("MMD=",MMD)

    
 # calculate bounds

for a in np.sort(D['alpha'].unique()):
    for s in [0.03]:#np.sort(D['sigma'].unique()):
        for e in [0.01]:#np.sort(D['epsilon'].unique()):         
            stratum = (D['sigma']==s)&(D['alpha']==a)&(D['epsilon']==e)#

            if(len(D[stratum])==0):
                print("empty stratum")
            #elif(len(D[stratum][])>)
            else:
                train_germain = D[stratum]['train_germain'].values
                e_s = D[stratum]['e_s'].values
                e_t = D[stratum]['e_t'].values
                d_tx = D[stratum]['d_tx'].values
                d_sx = D[stratum]['d_sx'].values
                KL = D[stratum]['KL'].values
                res, bestparam, boundparts = grid_search(train_germain,e_s,e_t,d_tx,d_sx,KL,delta,m_bound,m_target)
                res_inf, _, _ = grid_search(train_germain,e_s,e_t,d_tx,d_sx,KL,delta,1e16,1e16)
                
                
                if task==1 or task==4 or task==7:
                    bestparam2=[0,0] ## we cannot calculate the bound with no obvious overlap
                    res2=0
                    res2_inf=0
                else:
                    # calculate beta bound
                    [res2, bestparam2, boundparts2] = grid_search(train_germain,e_s,e_t,d_tx,d_sx,KL,delta,m_bound,m_target,beta_bound=True,beta_inf=beta_inf)
                    [res2_inf, _, _] = grid_search(train_germain,e_s,e_t,d_tx,d_sx,KL,delta,1e16,1e16,beta_bound=True,beta_inf=beta_inf)
                # calculate mmd bound
                ##
                
                res3,bestparam3, boundparts3 = grid_search_single(train_germain,KL,delta,m_bound,MMD)    
                res3_inf,_,_ = grid_search_single(train_germain,KL,delta,1e16,MMD)    
                    
                ## save the results
                D.loc[stratum,'germain_bound'] = res
                D.loc[stratum,'germain_bound_inf'] = res_inf
                D.loc[stratum,'beta_bound'] = res2
                D.loc[stratum,'beta_bound_inf'] = res2_inf
                D.loc[stratum,'mmd_bound'] = res3
                D.loc[stratum,'mmd_bound_inf'] = res3_inf
                A=np.zeros(len(res))
                if len(res)<5:
                    A=-1
                else:
                    A[0]=bestparam[0]
                    A[1]=bestparam[1]
                    A[2]=bestparam2[0]
                    A[3]=bestparam2[1]
                    A[4]=bestparam3
                # saves disrho bound params first and then beta bound; a,omega
                D.loc[stratum,'params'] = A



In [ ]:
import sys
stratum = (D['sigma']==0.03)&(D['alpha']==0.3)&(D['epsilon']==0.01)
print(D[stratum]['germain_bound'])
sys.exit(-1)

#### here we want to find the minimum bound for each epsilon; for each value of alpha

res=[] # smallest eps
res2=[] # mid eps
res3=[] # largest eps
k=0
for e in np.sort(D['epsilon'].unique()):
    print(e)
    for a in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:#np.sort(D['alpha'].unique()):
        for s in [0.03]:#np.sort(D['sigma'].unique()):
        
            stratum = (D['sigma']==s)&(D['alpha']==a)&(D['epsilon']==e)
            
            gb = np.array(D[stratum]['germain_bound'])
            bb = np.array(D[stratum]['beta_bound'])       
            mb = np.array(D[stratum]['mmd_bound'])
            
            min_gb=gb.min()
            min_bb=bb.min()
            min_mb=mb.min()
            if k==0:
                res.append([min_gb,min_bb,min_mb])
            elif k==1:
                res2.append([min_gb,min_bb,min_mb])
            else:
                res3.append([min_gb,min_bb,min_mb])
    k+=1
print(res)
print(res)
print(res3)
alphas=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]


plt.plot(alphas,res,lw=2)
plt.plot(alphas,res2,lw=2)
plt.plot(alphas,res3,lw=2)
plt.ylabel("Minima of bound")
plt.xlabel(r"$\alpha$")
plt.legend(["add, $\epsilon=0.001$","mult, $\epsilon=0.001$","MMD, $\epsilon=0.001$","add, $\epsilon=0.01$","mult, $\epsilon=0.01$","MMD, $\epsilon=0.01$","add, $\epsilon=0.03$","mult, $\epsilon=0.03$","MMD, $\epsilon=0.03$"])
plt.savefig("alphaplotmnist.pdf")

In [ ]:
##### Here we should take the lowest part of the bound and the target error at that point

def find_minindex(minima,df):
    ## stupid function as all the indices are lost when saving results somehow
    j=0
    idx=0
    for i in df:
                if np.abs(i-minima)<=1e-4:
                    idx=j
                    break
                j+=1
    return idx
plt.rc('font', size=12, family='serif')
for a in [0]:#np.sort(D['alpha'].unique()):#[0]:#
    for s in [0.03]:#np.sort(D['sigma'].unique()):
        for e in [0.1]:#np.sort(D['epsilon'].unique()):
            stratum = (D['sigma']==s)&(D['alpha']==a)&(D['epsilon']==e)
            
            gb = D[stratum]['germain_bound']
            bb = D[stratum]['beta_bound']       
            mb = D[stratum]['mmd_bound']
            
            min_gb=gb.min()
            idx=find_minindex(min_gb,gb)
            w_gb=np.array(D[stratum]['target_germain'])[idx]
            
            min_bb=bb.min()
            idx=find_minindex(min_bb,bb)
            w_bb=np.array(D[stratum]['target_germain'])[idx]
            
            min_mb=mb.min()
            idx=find_minindex(min_mb,mb)
            w_mb=np.array(D[stratum]['target_germain'])[idx]
            
            data1 = [w_gb, w_bb, w_mb]
            data2 = [min_gb, min_bb, min_mb]
            width =0.3
            labels = ['Add.', 'Mult.', 'MMD']
            #### no title!
#             if task==6:
#                 #plt.title("CheXpert + ChestX-ray14 Mix, $\sigma_p=0.03$")
#             elif task==2:
#                # plt.title("MNIST + MNIST-M Mix, $\sigma_p=0.03$")
            plt.ylabel("Error")
            plt.xticks(range(len(data1)), labels)
            plt.axhspan(0.5, 1, color='lightgray', alpha=0.5, lw=0,hatch="/")
            plt.axhspan(1, max(max(data1),max(data2))+0.2, color='darkgray', alpha=0.5, lw=0, hatch="/")
            plt.bar(np.arange(len(data1)), data1, width=width)
            plt.bar(np.arange(len(data2))+ width, data2, width=width)
            
            plt.legend(["Worse than random","Uninformative region","Target error","Minimum of bound"],loc=2)
            plt.savefig("newintrochex.pdf")
            #plt.show()

In [ ]:
####### plot the bound one epoch into training the posterior for different epsilon to show what the 

In [ ]:

#### find best parameters and plot bounds at that point
best_gb=100
best_bb=100
best_mb=100
plt.rc('font', size=12, family='serif')
for a in np.sort(D['alpha'].unique()):
    for s in [0.03]:
        for e in [0.1]:#np.sort(D['epsilon'].unique()):#[0.01]:
            stratum = (D['sigma']==s)&(D['alpha']==a)&(D['epsilon']==e)
            
            gb = D[stratum]['germain_bound']
            bb = D[stratum]['beta_bound']       
            mb = D[stratum]['mmd_bound']
            
            min_gb=gb.min()
            min_bb=bb.min()
            min_mb=mb.min()
            
            if min_gb<best_gb:
                best_gb=min_gb
                idx=find_minindex(min_bb,bb)
                w_bb=np.array(D[stratum]['target_germain'])[idx]
                gbparam=[a,e,s]
        
            if min_bb<best_bb:
                best_bb=min_bb
                idx=find_minindex(min_bb,bb)
                w_bb=np.array(D[stratum]['target_germain'])[idx]
                bbparam=[a,e,s]
            if min_mb<best_mb:
                best_mb=min_mb
                idx=find_minindex(min_bb,bb)
                w_bb=np.array(D[stratum]['target_germain'])[idx]
                mbparam=[a,e,s]
data1 = [w_gb, w_bb, w_mb]
data2 = [best_gb, best_bb, best_mb]
width =0.3
labels = ['Add.', 'Mult.', 'MMD']

plt.ylabel("Error")
plt.xticks(range(len(data1)), labels)
plt.axhspan(0.5, 1, color='lightgray', alpha=0.5, lw=0,hatch="/")
plt.axhspan(1, max(max(data1),max(data2))+0.2, color='darkgray', alpha=0.5, lw=0, hatch="/")
plt.bar(np.arange(len(data1)), data1, width=width)
plt.bar(np.arange(len(data2))+ width, data2, width=width)

plt.legend(["Worse than random","Uninformative region","Target error","Minimum of bound"],loc=2)
#plt.show()
# if task==2:
#     plt.savefig("bestmnist.pdf")
# elif task==6:
#     plt.savefig("newbestchex.pdf")

print(best_gb)
print(gbparam)

print(best_bb)
print(bbparam)

print(best_mb)
print(mbparam)

In [ ]:
#plt.rc('font', size=16, family='serif')

#sys.exit(-1)

####### plot of bound over training,

plt.rc('font', size=12, family='serif')
for a in [0,0.3]:#np.sort(D['alpha'].unique()):
    for s in [0.03]:
        for e in [0.1]:
            print('Sigma = %.5f' % s, 'Alpha = %.5f' % a,'Epsilon = %.5f' % e)
            
            ## pick out and load the relevant prior results
            r_files = [f for f in os.listdir(results_folder) if (str(int(a*100))+'_prior_results.pkl' in f)] 
            D_pr = pd.read_pickle(os.path.join(results_folder, r_files[0]))

            #get the num weightupdates to add to posterior count
            prior_w=D_pr['Weightupdates'].max()
            #print(D_pr['Weightupdates'])
            prior_source=D_pr['Trainerror']
            prior_target=D_pr['targeterror']
            
            stratum = (D['sigma']==s)&(D['alpha']==a)&(D['epsilon']==e)
            
            ws = D[stratum]['Weightupdates']
            #print(ws)
            
            #### if there are more than one weightupdate for one point for some reason
            #### we should remove the smaller in error
            if a!=0:
                ws+=prior_w ### add weightupdates from prior
            e_so = D[stratum]['train_germain']
            e_ta = D[stratum]['target_germain']
            e_so_std = D[stratum]['error_std']
            e_ta_std = D[stratum]['target_error_std']
            e_t= D[stratum]['e_t']
            e_s= D[stratum]['e_s']
            d_tx= D[stratum]['d_tx']
            d_sx= D[stratum]['d_sx']
            KL = D[stratum]['KL']
            gb = D[stratum]['germain_bound']
            bb = D[stratum]['beta_bound']       
            mb = D[stratum]['mmd_bound'] 
            
#             plt.figure(figsize=(9,6))
#             plt.grid(zorder=-100)
            
#             ### add prior source and target
#             #plt.plot(D_pr['Weightupdates'],prior_source , label='Prior source error', lw=2, c='b')
#             #plt.plot(D_pr['Weightupdates'],prior_target, label='Prior target error', lw=2, c='r')
            
#             plt.plot(ws, e_so, label='Source error', lw=2, c='b')
#             plt.fill_between(ws, e_so-e_so_std, e_so+e_so_std, alpha=0.5, zorder=0, color='b')

#             plt.plot(ws, e_ta, label='Target error', lw=2, c='r')
#             plt.fill_between(ws, e_ta-e_ta_std, e_ta+e_ta_std, alpha=0.5, zorder=0, color='r')

#             plt.plot(ws, gb, label='Germain bound', lw=2, c='g')
#             plt.plot(ws, bb, label='Beta bound', lw=2, c='k')
#             plt.plot(ws, mb, label='MMD bound', lw=2, c='m')
#             plt.xlim(0, 50000)
#             plt.ylim(0, 1.)
#             plt.legend()
#             plt.show()
            
            ### plot whole bound 
            plt.clf()
            plt.figure(figsize=(9,6))
            plt.grid(zorder=-100)
            if a!=0:
                ### add prior source and target
                plt.plot(D_pr['Weightupdates'],prior_source ,  lw=2, c='b')#label='Prior source error'
                plt.plot(D_pr['Weightupdates'],prior_target,  lw=2, c='r')#label='Prior target error',
            
            #plt.rc('font', size=12, family='serif')
            plt.plot(ws, e_so, label='Source error', lw=2, c='b')
            plt.fill_between(ws, e_so-e_so_std, e_so+e_so_std, alpha=0.5, zorder=100, color='b')

            plt.plot(ws, e_ta, label='Target error', lw=2, c='r')
            plt.fill_between(ws, e_ta-e_ta_std, e_ta+e_ta_std, alpha=0.5, zorder=100, color='r')

            plt.plot(ws, gb, label=r'Additive bound', lw=2, c='g')
            plt.plot(ws, bb, label='Multiplicative bound', lw=2, c='k')
            plt.plot(ws, mb, label='MMD bound', lw=2, c='m')
            #plt.xlim(-100, 60000)
            #plt.ylim(0, 2.5)
            plt.xlabel("Weight updates")
            plt.ylabel("Error")
            #if a==0:
            #    plt.title(r'No data, $\alpha=0$')
            #else:
            #    plt.title(r'$\alpha=$'+str(a))
            plt.legend()
            plt.show()
            #if a!=0:
                #plt.savefig("newchexresult_"+str(a)+".pdf")
                ######plt.savefig("newchexresult_"+str(a)+".pdf")



            

In [ ]:

#plot_amount_of_bound
plt.rc('font', size=12, family='serif')
for a in [0,0.3]:#np.sort(D['alpha'].unique()):
    for s in [0.03]:
        for e in [0.1]:
            print('Sigma = %.5f' % s, 'Alpha = %.5f' % a,'Epsilon = %.5f' % e)

            stratum = (D['sigma']==s)&(D['alpha']==a)&(D['epsilon']==e)
            ## pick out and load the relevant prior results
            r_files = [f for f in os.listdir(results_folder) if (str(int(a*100))+'_prior_results.pkl' in f)] 
            D_pr = pd.read_pickle(os.path.join(results_folder, r_files[0]))

            #get the num weightupdates to add to posterior count
            prior_w=D_pr['Weightupdates'].max()
            
            ws = D[stratum]['Weightupdates']
            if a!=0:
                ws+=prior_w
            e_so = D[stratum]['train_germain']
            e_ta = D[stratum]['target_germain']
            e_so_std = D[stratum]['error_std']
            e_ta_std = D[stratum]['target_error_std']
            e_t= D[stratum]['e_t']
            e_s= D[stratum]['e_s']
            d_tx= D[stratum]['d_tx']
            d_sx= D[stratum]['d_sx']
            KL = D[stratum]['KL']
            gb = D[stratum]['germain_bound']
            bb = D[stratum]['beta_bound']       
            mb = D[stratum]['mmd_bound'] 
            
            #just calculate the bound for each stratum and then plot the resulting proportions in a plot
        #### disrho
            
            
            m=D[stratum]['m_bound'].values[0]
            m_t=D[stratum]['m_target'].values[0]
            alp=D[stratum]['params'].values[0]
            omega=D[stratum]['params'].values[1]
            L=len(D[stratum])
            bound=[]
            aprime=2*alp/(1-np.exp(-2*alp))
            omegaprime=omega/(1-np.exp(-omega))
            a1=np.zeros(L)
            a2=np.zeros(L)
            a3=np.zeros(L)
            a4=np.zeros(L)
            a5=np.zeros(L)
            for i in range(L):
                dis_rho=np.abs(e_t.values[i]-e_s.values[i])
                lambda_rho=np.abs(d_tx.values[i]-d_sx.values[i])
                a1[i]=omegaprime*e_so.values[i]
                a2[i]=aprime/2*(dis_rho)
                a3[i]=(omegaprime/omega+aprime/alp)*(KL.values[i]+np.log(3/delta))/m
                a4[i]=lambda_rho
                a5[i]=(aprime-1)/2
                bound.append(a1[i]+a2[i]+a3[i]+a4[i]+a5[i])
            plt.clf()
            plt.grid(zorder=-100)
            plt.plot(ws,a1/bound)
            plt.plot(ws,a2/bound)
            plt.plot(ws,a3/bound)
            plt.plot(ws,a4/bound)
            plt.plot(ws,a5/bound)
            plt.xlabel("Weight updates")
            plt.ylabel("Fraction of bound")
            plt.legend(["Source error",r'Dis$_\rho$','KL',r'$\lambda_\rho$',"Constant"])
            #plt.show()
            plt.savefig("newdisrho_portion_"+str(a)+".pdf")
            #plt.savefig("chexdisrho_portion_"+str(a)+".pdf")
            #sys.exit(-1)
        ### beta
            plt.clf()
            plt.grid(zorder=-100)
            #plt.rc('font', size=12, family='serif')
            #plt.clearfig
            BETA=beta_inf
            beta_bound=[]
            b=D[stratum]['params'].values[2]
            c=D[stratum]['params'].values[3]
            bprime=BETA*b/(1-np.exp(-b))
            cprime=c/(1-np.exp(-c))
            a1=np.zeros(L)
            a2=np.zeros(L)
            a3=np.zeros(L)
            for i in range(L):
                a1[i]=cprime/2*(d_tx.values[i])
                a2[i]=bprime*e_s.values[i]
                a3[i]=(cprime/(m_t*c)+bprime/(m*b))*(2*KL.values[i]+np.log(2/delta))
                beta_bound.append(a1[i]+a2[i]+a3[i])
            plt.plot(ws,a2/beta_bound)
            plt.plot(ws,a1/beta_bound)
            plt.plot(ws,a3/beta_bound)
            plt.xlabel("Weight updates")
            plt.ylabel("Fraction of bound")
            plt.legend([r'$e_s$',r'$d_{tx}$','KL'])
            #plt.show()
            plt.savefig("newbeta_portion_"+str(a)+".pdf")
            #plt.savefig("chexbeta_portion_"+str(a)+".pdf")
        ### MMD
            plt.clf()
            plt.grid(zorder=-100)
            #plt.rc('font', size=12, family='serif')
            #plt.clearfig
            
            mmd_bound=[]
           
            betaparam=D[stratum]['params'].values[4]
            
            a1=np.zeros(L)
            a2=np.zeros(L)
            a3=np.zeros(L)
            beta_inv=(1-betaparam)


            for i in range(L):
                a1[i]=e_so.values[i]/betaparam
                a2[i]=(KL.values[i]+np.log(1/delta))/(2*betaparam*beta_inv*m)
                a3[i]=MMD ## fixed for now
                mmd_bound.append(a1[i]+a2[i]+a3[i])
            plt.plot(ws,a1/mmd_bound)
            plt.plot(ws,a3/mmd_bound)
            plt.plot(ws,a2/mmd_bound)
            plt.xlabel("Weight updates")
            plt.ylabel("Fraction of bound")
            plt.legend(['Source error','MMD','KL'])
            #plt.show()
            plt.savefig("newmmd_portion_"+str(a)+".pdf")
            #plt.savefig("chexmmd_portion_"+str(a)+".pdf")
            ##### end of proportions plot
         

In [ ]:
"""
Plot the influence of varying alpha by having the 
"""
if task==2:
    beta_inf=10.986111
    MMD=1.41
elif task==6:
    beta_inf=4
    MMD=1.41 ##? 
plt.rc('font', size=12, family='serif')
##### we calculate after the first epoch trained on the posterior, i.e. 2_1

for e in np.sort(D['epsilon'].unique()):
    beta_bound=[]
    disrho_bound=[]
    mmd_bound=[]
    for s in [0.03]:
        for a in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:#np.sort(D['alpha'].unique()):#np.sort(D['epsilon'].unique()):
            print('Sigma = %.5f' % s, 'Alpha = %.5f' % a,'Epsilon = %.5f' % e)

            stratum = (D['sigma']==s)&(D['alpha']==a)&(D['epsilon']==e)&(D["checkpoint"]=="2_1")
          
           
            
            ## compute the bound with all parameters =1/2 for consistency
            
            e_so = D[stratum]['train_germain']
            e_ta = D[stratum]['target_germain']
            e_so_std = D[stratum]['error_std']
            e_ta_std = D[stratum]['target_error_std']
            e_t= D[stratum]['e_t']
            e_s= D[stratum]['e_s']
            d_tx= D[stratum]['d_tx']
            d_sx= D[stratum]['d_sx']
            KL = D[stratum]['KL']
            
            
            #just calculate the bound for each stratum and then plot the resulting proportions in a plot
        #### disrho
            
            
            m=D[stratum]['m_bound'].values[0]
            m_t=D[stratum]['m_target'].values[0]
            alp=1/2
            omega=1/2
           
            aprime=2*alp/(1-np.exp(-2*alp))
            omegaprime=omega/(1-np.exp(-omega))
            
            dis_rho=np.abs(e_t.values[0]-e_s.values[0])
            lambda_rho=np.abs(d_tx.values[0]-d_sx.values[0])
            a1=omegaprime*e_so.values[0]
            a2=aprime/2*(dis_rho)
            a3=(omegaprime/(omega)+aprime/(alp))*(KL.values[0]+np.log(3/delta))/m
            a4=lambda_rho
            a5=(aprime-1)/2
            disrho_bound.append(a1+a2+a3+a4+a5)
            #print(disrhobound)
        
            
            
        ### beta
            BETA=beta_inf
           
            b=1/2
            c=1/2
            bprime=BETA*b/(1-np.exp(-b))
            cprime=c/(1-np.exp(-c))
            
            a1=cprime/2*(d_tx.values[0])
            a2=bprime*e_s.values[0]
            a3=(cprime/(m_t*c)+bprime/(m*b))*(2*KL.values[0]+np.log(2/delta))
            beta_bound.append(a1+a2+a3)
            #print(beta_bound)
            ### MMD
            
           
            betaparam=1/2
        
            beta_inv=(1-betaparam)


            a1=e_so.values[0]/betaparam
            a2=(KL.values[0]+np.log(1/delta))/(2*betaparam*beta_inv*m)
            a3=MMD ## fixed for now
            mmd_bound.append(a1+a2+a3)
    plt.xlabel(r"$\alpha$")
    plt.ylabel("Bound value")
    plt.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],disrho_bound)
    #plt.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],beta_bound)
    #plt.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],mmd_bound)
plt.legend([r"$\epsilon$=0.15",r"$\epsilon$=0.1",r"$\epsilon$=0.05"]) ### change for other task
#plt.savefig("mnistalphaplot_disrho.pdf")
#plt.savefig("chexalphadisrho.pdf")
#plt.show()

In [ ]:
plt.rc('font', size=16, family='serif')
display(D.head(5))
D = D.sort_values(by='Weightupdates')

#plt.plot(D[D['sigma']==.03]['Weightupdates'], D[D['sigma']==.03]['train_germain'])
for a in np.sort(D['alpha'].unique()):
    for s in np.sort(D['sigma'].unique()):
        print('Sigma = %.5f' % s, 'Alpha = %.5f' % a)

        stratum = (D['sigma']==s)&(D['alpha']==a)
        ws = D[stratum]['Weightupdates']
        e_so = D[stratum]['train_germain']
        e_ta = D[stratum]['target_germain']
        e_so_std = D[stratum]['error_std']
        e_ta_std = D[stratum]['target_error_std']
        KL = D[stratum]['KL']
        gb = D[stratum]['germain_bound']
        gbi = D[stratum]['germain_bound_inf']
        bb = D[stratum]['beta_bound']       
        bbi = D[stratum]['beta_bound_inf']       

        plt.figure(figsize=(9,6))
        plt.grid(zorder=-100)

        plt.plot(ws, e_so, label='Source error', lw=2)    
        plt.plot(ws, e_ta, label='Target error', lw=2)    
        plt.plot(ws, gb, label='Germain bound', lw=2)
        plt.plot(ws, gbi, label='Germain bound (m=inf)', lw=2)
        plt.plot(ws, bb, label='Beta bound', lw=2)
        plt.plot(ws, bbi, label='Beta bound (m=inf)', lw=2)
        plt.xlim(0, 8000)
        plt.ylim(0, 1)

        plt.legend()
        plt.show()